In [1]:
!pip install pymorphy2

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2
import plotly.graph_objs as go
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

#векторизация
from sklearn.feature_extraction.text import TfidfVectorizer

# #выделение тематик
# from sklearn.decomposition import LatentDirichletAllocation
# from gensim.corpora.dictionary import Dictionary
# from numpy import array
# import pyLDAvis
# import pyLDAvis.gensim_models as gensimvis
# import gensim

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_excel('/content/drive/MyDrive/ML ITHUB осень/ДИ 2023/lessons/data/vacancies.xlsx')

In [4]:
df.shape

(2706, 3)

In [5]:
%%time
text=df['description']
norm_text=[]
morph=pymorphy2.MorphAnalyzer()
for s in text:
    s1 = re.sub(r'[^\w\s]+|[\d]+', r'',s).strip()
    s1 = s1.lower()
    s1 = word_tokenize(s1)
    words=[]
    for i in s1:
        pv = morph.parse(i)
        words.append(pv[0].normal_form)
    sentence=' '.join(words)
    norm_text.append(sentence)

CPU times: user 46 s, sys: 198 ms, total: 46.2 s
Wall time: 1min


In [6]:
russian_stopwords = stopwords.words("russian")
vectorizer_tfidf = TfidfVectorizer(max_features=500, min_df=20, max_df=0.7, stop_words=russian_stopwords)
text_tfidf = vectorizer_tfidf.fit_transform(norm_text)
text_tfidf = pd.DataFrame(text_tfidf.toarray(),columns=vectorizer_tfidf.get_feature_names_out())
text_tfidf.head()

,ajax,api,aspnet,backend,bitrix,bootstrap,cms,crm,css,delphi,...,читать,чпу,чтение,чужой,шаблон,эксплуатация,электронный,это,являться,язык
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046909


## Понижение размерности и визуализация

In [ ]:
!pip install umap-learn

#понижение размерности
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.4-py3-none-any.whl size=86770 sha256=5b01cc8c7956aafb00d111b415a5dd8aaa5e4c515f5db7d7b16e551b14c46098
  Stored in directory: /root/.cache/pip/wheels/fb/66/29/199acf5784d0f7b8add6d466175ab45506c96e386ed5dd0633
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55615 sha256=9a75e896d5dba65ba4888bbd76d2f602f4790f4ee2448b5a951f0389a49a0c9a
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [ ]:
def show_fig(new_shape):

    fig = go.Figure(go.Scatter3d(x=new_shape[:, 0],
                        y=new_shape[:, 1],
                        z=new_shape[:, 2],
                        marker=dict(opacity=0.9,
                                    reversescale=True,
                                    colorscale='Blues',
                                    size=5),
                        line=dict (width=0.02),
                        mode='markers',
                                 text=df['title']))
    fig.update_layout(
        scene = dict(
            xaxis = dict(nticks=4),
                         yaxis = dict(nticks=4),
                         zaxis = dict(nticks=4),),
        width=700,
        margin=dict(r=20, l=10, b=10, t=10))
    fig.show()

In [ ]:
pca=PCA(n_components=3)
reduced_pca = pca.fit_transform(text_tfidf)
show_fig(reduced_pca)

In [ ]:
tsne=TSNE(n_components=3)
reduced_tsne = tsne.fit_transform(text_tfidf)
show_fig(reduced_tsne)

In [ ]:
import umap
umap_emb = umap.UMAP(n_components=3)
reduced_umap = umap_emb.fit_transform(text_tfidf)
show_fig(reduced_umap)

## Выделение тематик

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(text_tfidf)
for i,topic in enumerate(LDA.components_):
    print(f'5 слов для темы #{i}:')
    print([vectorizer_tfidf.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print('\n')

5 слов для темы #0:
['программирование', 'разработка', 'предприятие', 'знание', 'работа', 'типовой', 'опыт', 'доработка', 'платформа', 'конфигурация']


5 слов для темы #1:
['mysql', 'javascript', 'работа', 'опыт', 'разработка', 'css', 'знание', 'html', 'php', 'сайт']


5 слов для темы #2:
['станок', 'микроконтроллер', 'высокий', 'групповой', 'тренировка', 'опыт', 'проведение', 'образование', 'работа', 'программа']


5 слов для темы #3:
['обеспечение', 'язык', 'sql', 'система', 'данные', 'программный', 'опыт', 'работа', 'знание', 'разработка']


5 слов для темы #4:
['направление', 'программа', 'работа', 'программирование', 'развиваться', 'образование', 'обучение', 'технический', 'ребёнок', 'желание']




In [7]:
!pip install --upgrade gensim

In [8]:
text_clean= []
for row in norm_text:
        text_clean.append(row.split())

from gensim.models import Phrases
bigram = Phrases(text_clean)
trigram = Phrases(bigram[text_clean])

for idx in range(len(text_clean)):
    for token in bigram[text_clean[idx]]:
        if '_' in token:

            text_clean[idx].append(token)
    for token in trigram[text_clean[idx]]:
        if '_' in token:
          text_clean[idx].append(token)

In [9]:
from gensim.corpora.dictionary import Dictionary
from numpy import array
dictionary = Dictionary(text_clean)
dictionary.filter_extremes(no_below=10, no_above=0.1)
corpus = [dictionary.doc2bow(doc) for doc in text_clean]
print('Количество уникальных токенов: %d' % len(dictionary))
print('Количество документов: %d' % len(corpus))

Количество уникальных токенов: 1589
Количество документов: 2706


In [10]:
from gensim.models.ldamulticore import LdaMulticore
model=LdaMulticore(corpus=corpus,id2word=dictionary, num_topics=2)
model.show_topics()

[(0,
  '0.009*"умение_работать" + 0.007*"база_данные" + 0.006*"на_платформа" + 0.006*"в_область" + 0.006*"программный_обеспечение" + 0.006*"программный_продукт" + 0.005*"язык_программирование" + 0.005*"информационный_система" + 0.004*"быть" + 0.004*"управление"'),
 (1,
  '0.010*"программный_обеспечение" + 0.007*"на_платформа" + 0.007*"программный_продукт" + 0.006*"информационный_система" + 0.006*"база_данные" + 0.006*"язык_программирование" + 0.006*"типовой_конфигурация" + 0.006*"высокий_технический" + 0.005*"html_css" + 0.005*"предприятие"')]

In [11]:
!pip install fastapi kaleido python-multipart uvicorn

In [15]:
!pip uninstall pandas
!pip install pandas==2.0.0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Found existing installation: pandas 2.2.1
Uninstalling pandas-2.2.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/pandas-2.2.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/pandas/*
Proceed (Y/n)? y
  Successfully uninstalled pandas-2.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 24.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.0 which is incompatible.


In [12]:
!pip install pyLDAvis

In [13]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import gensim
pyLDAvis.enable_notebook()
data = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(data)

/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1429: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


## ЛР NLP - сдаем 11.11

* ищем набор текстовых данных (берем только русскоязычный текст), минимум 1000 записей - в помощь kaggle и весь интернет
* для текстовых данных проводим токенизацию,лемматизацию избавляемся от спецсимволов и цифр
* для всего набора данных строим WordCloud - делаем выводы
* с помощью библиотеки updipe в зависимости от варианта (указан в столбце ЛР NLP) находим связку слов в наборе текстовых данных  и выводим топ-20 по популярности - делаем соответствующие выводы
* проводим векторизацию с помощью TF-IDF матрицы и понижаем размерность до 3 компонентов с помощью ВСЕХ трех методов (PCA,TSNE,UMAP)
* для каждого набора из трех компонентов строим 3х-мерный график - делаем выводы по каждому графику (выделяются ли отдельные тематики)
* занимаемся тематическим моделированием и пробуем выявить тематики ваших текстов с помощью LDA

P.S. если соберете данные для этой лабы с помощью самописного парсера (покажете его работу) - доп 3 балла по МЛ и доп 3 балла по МСОД

P.P.S +3 балла за свою модель w2v